# collect youtube ranking data and visualize

In [1]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--single-process')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [4]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver.get(url)
# time.sleep(2)

In [5]:
trs = driver.find_elements_by_tag_name('tr')
len(trs)

102

In [6]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [7]:
# soup.select('form > table > tbody > tr')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [8]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip()
category

'[음악/댄스/가수]'

In [9]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [10]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.view_cnt').text[:-1]
subscriber, view, video

('6400만', '190억0381만', '190억0381')

In [25]:
channels = []
for channel in channel_list:
  category = channel.select_one('p.category').get_text().strip(' \n[] ')
  name = channel.select_one('.subject a').text.strip()
  subscriber = channel.select_one('.subscriber_cnt').text
  view = channel.select_one('.view_cnt').text
  video = channel.select_one('.view_cnt').text[:-1]
  channels.append([category, name, subscriber, view, video])

In [26]:
df = pd.DataFrame(channels, columns=['category', 'name', 'subscriber', 'view', 'video'])
df.head()

,category,name,subscriber,view,video
0,음악/댄스/가수,May J Lee 메이제이 리,57만,2861만,2861
1,BJ/인물/연예인,덕자전성시대,56만,6116만,6116
2,음악/댄스/가수,SUGA - Topic,56만,1만,1
3,TV/방송,KBS Entertain: 깔깔티비,56만,7억9237만,7억9237
4,애완/반려동물,Arirang은 고양이들내가 주인,56만,1억6574만,1억6574


In [27]:
df.tail()

,category,name,subscriber,view,video
95,게임,미소,51만,2억1077만,2억1077
96,미분류,밉지않은 관종언니,51만,6774만,6774
97,미분류,복지마블TV [Welfare Marble],51만,4996만,4996
98,음식/요리/레시피,Muggo,51만,1억8372만,1억8372
99,게임,임선비,51만,2억3429만,2억3429


- func : 10 thousand, 1 million to number

In [28]:
def covert_unit(s):
  
  s = s.replace('억', '').replace('개', '').replace(',', '')
  s = s.replace('만', '0000')
  return f'{int(s):,d}'

In [29]:
covert_unit('10억6557만')

'1,065,570,000'

- second page

In [30]:
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()

In [31]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [32]:
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [37]:
results = []
for channel in channel_list:
  category = channel.select_one('p.category').get_text().strip(' \n[] ')
  name = channel.select_one('.subject a').text.strip()
  subscriber = channel.select_one('.subscriber_cnt').text
  view = channel.select_one('.view_cnt').text
  video = channel.select_one('.view_cnt').text[:-1]
  results.append([category, name, subscriber, view, video])

In [38]:
df = pd.DataFrame(results, columns=['category', 'name', 'subscriber', 'view', 'video'])
df.head()

,category,name,subscriber,view,video
0,음악/댄스/가수,May J Lee 메이제이 리,57만,2861만,2861
1,BJ/인물/연예인,덕자전성시대,56만,6116만,6116
2,음악/댄스/가수,SUGA - Topic,56만,1만,1
3,TV/방송,KBS Entertain: 깔깔티비,56만,7억9237만,7억9237
4,애완/반려동물,Arirang은 고양이들내가 주인,56만,1억6574만,1억6574


- page 1 - 10 crawling

In [39]:
results = []
for page in range(1,11):
  url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(page)
  driver.get(url)
  time.sleep(3)
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  channel_list = soup.select('.aos-init')

  for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[] ')
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').text
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.view_cnt').text[:-1]
    results.append([category, name, subscriber, view, video])

In [40]:
df = pd.DataFrame(results, columns=['category', 'name', 'subscriber', 'view', 'video'])
df.head()

,category,name,subscriber,view,video
0,음악/댄스/가수,BLACKPINK,6400만,190억0381만,190억0381
1,음악/댄스/가수,HYBE LABELS,6030만,187억1305만,187억1305
2,음악/댄스/가수,BANGTANTV,5640만,121억9496만,121억9496
3,음악/댄스/가수,SMTOWN,2850만,218억7092만,218억7092
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,110억5288


In [41]:
df.tail()

,category,name,subscriber,view,video
995,게임,미소,51만,2억1077만,2억1077
996,미분류,밉지않은 관종언니,51만,6774만,6774
997,미분류,복지마블TV [Welfare Marble],51만,4996만,4996
998,음식/요리/레시피,Muggo,51만,1억8372만,1억8372
999,게임,임선비,51만,2억3429만,2억3429


In [42]:
df.to_csv('youtube_rank.csv', index=False)

In [43]:
driver.close()